<h1>Importation des packages</h1>

In [1]:
import sys
sys.path.append('../')

import numpy as np
import warnings
warnings.filterwarnings("ignore")

import cv2

from fonctionsUtiles import functions as f
from fonctionsUtiles import architecture
from sklearn.model_selection import train_test_split

ImportError: No module named 'fonctionsUtiles'

In [11]:
from tensorflow import keras
from tensorflow.python.keras.applications import NASNetLarge, InceptionV3
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

<h1>Chargement de toutes les données</h1>

In [3]:
# Get training data from the Axionable track
X_axio = np.load('../Datasets/axionable_data/X_train_axio.npy')
Y_axio = np.load('../Datasets/axionable_data/Y_train_axio.npy')
print('Axionable data Loaded. Shape = ', np.shape(X_axio))

# Get training data from IronCar track
# New track - Double chicane
X_chicane = np.load('../Datasets/ironcar_data/new_track/x_chicane.npy')
Y_chicane = np.load('../Datasets/ironcar_data/new_track/y_chicane.npy')
print('Ironcar new track chicane Loaded. Shape = ', np.shape(X_chicane))

# Old track - Balanced dataset
X_iron = np.load('../Datasets/ironcar_data/old_track/balanced_iron_X.npy')
Y_iron = np.load('../Datasets/ironcar_data/old_track/balanced_iron_Y.npy')
print('Ironcar old track data Loaded. Shape = ', np.shape(X_iron))

Axionable data Loaded. Shape =  (26449, 90, 250, 3)
Ironcar new track chicane Loaded. Shape =  (1519, 90, 250, 3)
Ironcar old track data Loaded. Shape =  (16028, 90, 250, 3)


<h1>Resizing des données</h1>

In [4]:
def resizing(array):
    masterX = list()
    for arr in array:
        masterX.append(cv2.resize(arr, (331,331)))
    return np.array(masterX)

X_axio_resized = resizing(X_axio)
X_chicane_resized = resizing(X_chicane)
X_iron_resized = resizing(X_iron)

print('Axionable data new size. Shape = ', np.shape(X_axio_resized))
print('Ironcar new track chicane new size. Shape = ', np.shape(X_chicane_resized))
print('Ironcar old track data new size. Shape = ', np.shape(X_iron_resized))

Axionable data new size. Shape =  (26449, 331, 331, 3)
Ironcar new track chicane new size. Shape =  (1519, 331, 331, 3)
Ironcar old track data new size. Shape =  (16028, 331, 331, 3)


<h1>Séparation des données en Train, Test et Validation</h1>

In [5]:
# Dictionnaire qui stock quelques variables importantes
args = {"augmentation": True,
        "train_split": 0.8,
        "val_split":0.2,
        "test_split":0.2,
        "batch_size": 200,
        "epochs": 12,
        "early_stop":True,
        "patience":2}

In [6]:
# Concatenation de toutes les données chargées
X = np.concatenate((X_axio_resized, X_chicane_resized, X_iron_resized))
Y = np.concatenate((Y_axio, Y_chicane, Y_iron))
print('All data loaded and concatenated. Shape = ', np.shape(X))

# Suppression des variables tmp qui contiennent les images
del X_axio, Y_axio, X_chicane, Y_chicane, X_iron, Y_iron, X_axio_resized, X_chicane_resized, X_iron_resized

All data loaded and concatenated. Shape =  (43996, 331, 331, 3)


In [7]:
# Réparatition du jeu de données en train test 80 - 20 %
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12)

In [8]:
# Répartition du jeu d'entrainement en jeu d'entrainement et en jeu de validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=12)

In [9]:
print('All training data loaded and augmented. Shape = ', np.shape(X_train))
print('All validation data loaded and augmented. Shape = ', np.shape(X_val))
print('All testing data loaded and augmented. Shape = ', np.shape(X_test))

All training data loaded and augmented. Shape =  (28156, 331, 331, 3)
All validation data loaded and augmented. Shape =  (7040, 331, 331, 3)
All testing data loaded and augmented. Shape =  (8800, 331, 331, 3)


In [10]:
# Suppression des variables tmp qui contiennent les images
del X,Y

<h1>Augmentation des données</h1>

In [11]:
# Data augmentation of the dataset / Adjust the proportion of each transformation you want to apply.
if args['augmentation']:
    print('Augmenting data... Wait...')
    # Data augmentation 25% of random brightness.
    X_bright, Y_bright = f.generate_brightness(X_train, Y_train, proportion=0.25)
    # Data augmentation 25% of night effect.
    X_night, Y_night = f.generate_night_effect(X_train, Y_train, proportion=0.25)
    # Data augmentation 25% of horizontal flipping.
    X_flip, Y_flip = f.generate_horizontal_flip(X_train, Y_train, proportion=0.25)
    # Data augmentation 25% of random shadows.
    X_shadow, Y_shadow = f.generate_random_shadows(X_train, Y_train, proportion=0.25)
    # Data augmentation 25% of chained tranformations (bright + shadows + flip).
    X_chain, Y_chain = f.generate_chained_transformations(X_train, Y_train, proportion=0.25)

    # Concatenating Axionable dataset with the transformations.
    X_train = np.concatenate((X_train, X_bright, X_night,
                                X_flip, X_shadow, X_chain))

    Y_train = np.concatenate((Y_train, Y_bright, Y_night, 
                                Y_flip, Y_shadow, Y_chain)).astype('float32')

    print('Axionable data after augmentation. Shape = ', np.shape(X_train))

  0%|          | 24/7039 [00:00<00:29, 239.96it/s]

Augmenting data... Wait...


100%|██████████| 7039/7039 [00:34<00:00, 203.35it/s]


Axionable data after augmentation. Shape =  (63351, 331, 331, 3)


In [12]:
# Suppression des variables tmp qui contiennent les images
del X_bright, Y_bright, X_night, Y_night, X_flip, Y_flip, X_shadow, Y_shadow, X_chain, Y_chain

<h1>Construction du modèle</h1>

In [12]:
# Nombre de classes en output du modèle. Ici : hard left, left, forward, right et hard right
num_classes = 5

# Initialisation du modèle : on télécharge le modèle MobilNetV2 et on rajoute la dernière couche
# de classification propre à notre problème
# my_new_model = Sequential()
# my_new_model.add(InceptionV3(include_top=True, pooling='max', weights="imagenet"))
# my_new_model.add(Dense(128, activation='relu'))
# my_new_model.add(Dropout(0.2))
# my_new_model.add(Dense(128, activation='relu'))
# my_new_model.add(Dropout(0.2))
# my_new_model.add(Dense(num_classes, activation='softmax'))

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
l = Dense(128, activation = 'relu')(base_model.output)
l = Dropout(0.2)(l)
l = Dense(64, activation = 'relu')(l)
l = Dropout(0.2)(l)
l = Flatten()(l)
predictions = Dense(1, activation = 'linear')(l)
model = Model(inputs=base_model.input, outputs=predictions)

# On indique que les couches avant la couche de classification ne doivent pas être entrainées
# my_new_model.layers[0].trainable = False

In [15]:
# On vérfie la structure du modèle
model.summary()
print(len(model.layers))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 111, 111, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 111, 111, 32) 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 111, 111, 32) 0           batch_normalization_470[0][0]    
______________________________________________________________________________________________

In [15]:
# On définit la fonction d'optimisation du modèle
my_new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Paramètre de réduction du learning rate si l'accuracy en validation diminue
reduce_lr = ReduceLROnPlateau(monitor='loss', 
                              factor=0.2,
                              patience=5, 
                              min_lr=0.001)

hist = my_new_model.fit(
                X_train, 
                Y_train,
                nb_epoch= args['epochs'],
                batch_size=args['batch_size'], 
                verbose=1, 
                validation_data=(X_val, Y_val),
                shuffle=True, 
                callbacks = [reduce_lr])

my_new_model.save('TransferNASNetLarge_12Epochs_doubledenselayer.h5')

W0617 17:30:31.790341 139796909606656 training.py:618] The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 63351 samples, validate on 7040 samples
Epoch 1/12
  400/63351 [..............................] - ETA: 5:08:55 - loss: 8.2947 - acc: 0.2800